# Role Prompting 的局限性和 System 2 Thinking 的新范式

基于吴恩达 x OpenAI 提示词工程课程的实践 notebook

## 📚 什么是 Role Prompting？

**Role Prompting** 是给模型分配角色（"你是XX专家"），让它模仿该角色的行为和语言风格。

### ❌ Role Prompting 的局限性

虽然 Role Prompting 能改变模型的"说话方式"，但存在以下根本问题：

1. **缺乏自评能力**：单角色对错误答案同样自信，无法自我纠错
2. **幻觉累积**：多步推理场景下，角色描述无法动态更新，导致后续步骤建立在早期幻觉之上
3. **认知不一致**：多 Agent 系统里，同一角色被复制 N 份，出现逻辑冲突

**核心问题**：Role Prompting 只改变"说话方式"，不改变"思维深度"。

## 🧠 System 2 Thinking 范式

**System 2 Thinking** 源自心理学家 Daniel Kahneman 的双系统理论：

| 系统 | 特点 | LLM 对应 |
|------|------|----------|
| **System 1** | 快速、直觉、自动 | 传统 LLM 的即时响应 |
| **System 2** | 慢速、深思、分析 | 推理模型（o1/R1/Qwen3-thinking） |

### 🔬 两种实现方式

#### 1️⃣ 算法层面：System 2 Distillation（蒸馏）

**目标**：让模型"内化" System 2 能力，不需要每次都生成中间推理步骤

**工作原理**：
1. 用 System 2 提示（如 CoT）让模型完成任务
2. 验证答案正确性，保留正确的
3. **移除推理 token**，只保留问题和答案
4. 用这个数据集继续预训练/LoRA

**效果**：
- 性能达到或超过原始 System 2 方法
- 响应速度更快、计算成本更低（不需要生成中间步骤）

**类比**：就像人类学开车——刚开始需要有意识地思考每个步骤（System 2），熟练后变成下意识操作（System 1）。

#### 2️⃣ 应用层面：Multi-Agent Verification（多智能体验证）

通过多个 Agent 相互协作和检查，实现深度思考：

**策略 1：Multi-Agent System 2**
- 一个 Agent 生成答案
- 另一个 Agent 扮演批评者，指出问题
- 主 Agent 综合意见给出最终答案

**策略 2：分层验证**
```
Level 1：单模型自验证（最简单）
  → Thought → Answer → Self-Check → ✓/✗

Level 2：双模型交叉验证
  → Model A 生成 → Model B 审查 → 反馈修正

Level 3：Multi-Agent 验证（最强）
  → 主 Agent 分解任务 → 专项 Agent 验证各维度 → 汇总
```

## 🎯 本实验目标

通过对比实验展示：
1. **Role Prompting 的失败案例**
2. **System 2 单模型自验证**的改进
3. **Multi-Agent 验证**的最佳效果

## 🤖 测试模型

使用 **Qwen3 8B** 模型


---

## 第一步：环境准备

安装并启动 Ollama 服务

In [1]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time

print("⬇️ 正在安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行


In [2]:
# === 2. 下载模型并定义核心函数 ===
import requests
import json
import time

print("⬇️ 正在下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型下载完成！")

# 配置 Ollama API 地址
OLLAMA_URL = "http://localhost:11434"

def get_completion(prompt, model="qwen3:8b", temperature=0.7, show_output=True, enable_thinking=False, show_thinking=False):
    """
    调用 Qwen3 模型获取回复

    参数:
        prompt: 提示词
        model: 模型名称
        temperature: 温度参数 (0.0-1.0)
        show_output: 是否显示输出
        enable_thinking: 是否开启 Qwen3 内置思考模式（默认 False）
        show_thinking: 是否显示思考内容（仅当 enable_thinking=True 时有效）

    返回:
        模型的回复文本
    """
    url = f"{OLLAMA_URL}/api/chat"
    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "num_ctx": 8192,
        "num_predict": 4096,
        "repeat_penalty": 1.07
    }

    messages = [{"role": "user", "content": prompt}]

    # 构建请求payload - 关键：显式传 think 参数
    payload = {
        "model": model,
        "messages": messages,
        "stream": True,
        "options": options,
        "think": enable_thinking  # 控制思考模式
    }

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()

        result = ""
        thinking_text = ""
        is_thinking_shown = False

        if show_output:
            print("🤖 模型回复: ", end='', flush=True)

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))

                    if 'message' in data:
                        # 处理 thinking 字段
                        if 'thinking' in data['message'] and data['message']['thinking']:
                            thinking_content = data['message']['thinking']
                            thinking_text += thinking_content

                            if show_output and show_thinking and not is_thinking_shown:
                                print(f"\n💭 [思考过程]\n{'─'*80}", flush=True)
                                is_thinking_shown = True

                            if show_output and show_thinking:
                                print(thinking_content, end='', flush=True)

                        # 处理 content 字段
                        if 'content' in data['message'] and data['message']['content']:
                            content = data['message']['content']

                            if show_output:
                                if is_thinking_shown:
                                    print(f"\n{'─'*80}\n✅ [正式回答]: ", flush=True)
                                    is_thinking_shown = False
                                print(content, end='', flush=True)

                            result += content

                    if data.get('done', False):
                        if show_output:
                            print()
                        break

                except json.JSONDecodeError:
                    continue

        # 如果 content 为空但 thinking 有内容
        if not result.strip() and thinking_text.strip():
            if show_output:
                print(f"⚠️ content 为空，检测到 thinking 内容（{len(thinking_text)} 字符）")
            result = f"[思考内容]: {thinking_text[:500]}..." if len(thinking_text) > 500 else f"[思考内容]: {thinking_text}"

        return result.strip()

    except Exception as e:
        if show_output:
            print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

print("✅ 核心函数加载完成")
print("💡 可用函数: get_completion(prompt, enable_thinking=False, show_thinking=False)")
print("   - enable_thinking: 开启/关闭 Qwen3 内置思考")
print("   - show_thinking: 是否显示思考过程")

⬇️ 正在下载 qwen3:8b...
✅ 模型下载完成！
✅ 核心函数加载完成
💡 可用函数: get_completion(prompt, enable_thinking=False, show_thinking=False)
   - enable_thinking: 开启/关闭 Qwen3 内置思考
   - show_thinking: 是否显示思考过程


---

# 实验一：Role Prompting 的局限性

## 📖 任务说明

**任务**：判断一个数学应用题的答案是否正确

**测试问题**：
> 一个班级有36个学生，其中男生比女生多4人。
> 请问这个班级有多少女生？

**学生答案**：18人（这个答案是错的！）

**正确答案**：16人

**陷阱分析**：
- 很多人看到"36个学生"会立刻想到 36÷2=18
- 这是**锚定效应**（Anchoring Bias）：被初始数字锚定，忽略了"男生比女生多4人"这个条件
- 正确做法：设女生x人，则男生(x+4)人，x + (x+4) = 36，解得x=16

## 🎯 实验设计

我们将对比三种方法：

1. **❌ 纯 Role Prompting**：让模型扮演"数学专家"，直接判断
2. **⚠️ Role + 简单验证**：角色 + 要求"仔细检查"
3. **✅ System 2 自验证**：强制先独立推理，再对比学生答案

## 💡 预期结果

- 方法1：可能被锚定到18，判断为正确（锚定效应）
- 方法2：仍然可能出错，因为"仔细检查"是模糊指令
- 方法3：强制列方程求解，避免被误导

In [3]:
# 实验一：Role Prompting 的局限性

# 测试问题和学生答案（学生答案是错的！）
logic_problem = """
一个班级有36个学生，其中男生比女生多4人。
请问这个班级有多少女生？
"""

student_answer = "18人"
correct_answer = "16人"

print("="*80)
print("🔬 实验一：Role Prompting 的局限性")
print("="*80)
print(f"测试问题：{logic_problem.strip()}")
print(f"\n学生答案：{student_answer}")
print(f"⚠️ 实际正确答案：{correct_answer}")
print(f"💡 陷阱：很多人会直接 36÷2=18，忽略了'男生比女生多4人'这个条件")
print("="*80)

# 方法1：❌ 纯 Role Prompting（最容易出错）
print("\n❌ 方法1：纯 Role Prompting（扮演角色直接判断）")
print("─"*80)
role_only_prompt = f"""你是一位数学专家，擅长分析数学应用题。

请判断以下学生答案是否正确：

问题：
{logic_problem}

学生答案：{student_answer}

判断："""

result1 = get_completion(role_only_prompt, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result1 or '不对' in result1 or '16' in result1 else '✗ 被误导'}")

time.sleep(3)

# 方法2：⚠️ Role + 简单验证
print("\n\n⚠️ 方法2：Role + 简单验证（加入\"仔细检查\"指令）")
print("─"*80)
role_check_prompt = f"""你是一位数学专家，擅长分析数学应用题。

请判断以下学生答案是否正确，并仔细检查你的判断。

问题：
{logic_problem}

学生答案：{student_answer}

请仔细检查后给出判断："""

result2 = get_completion(role_check_prompt, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result2 or '不对' in result2 or '16' in result2 else '✗ 被误导'}")

time.sleep(3)

# 方法3：✅ System 2 自验证（正确做法）
print("\n\n✅ 方法3：System 2 自验证（先独立推理，再对比）")
print("─"*80)
system2_prompt = f"""你的任务是判断学生的答案是否正确。

**重要：请严格按照以下步骤操作！**

步骤1：独立推理
- 忽略学生答案
- 自己从头开始分析这个数学问题
- 设未知数，列方程求解
- 得出你自己的答案

步骤2：对比验证
- 将你的答案与学生答案对比
- 如果不同，明确指出错误

步骤3：给出判断
- 输出：学生答案是否正确
- 理由：详细说明

---

问题：
{logic_problem}

学生答案：{student_answer}

请开始："""

result3 = get_completion(system2_prompt, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result3 or '不对' in result3 or '16' in result3 else '✗ 被误导'}")

print("\n" + "="*80)
print("💡 分析：")
print("  - 纯 Role Prompting：只改变说话风格，容易被锚定到18（直接平分）")
print("  - Role + 简单验证：'仔细检查'过于模糊，效果有限")
print("  - System 2 自验证：强制独立推理，列方程求解，避免被误导")
print("="*80 + "\n")

🔬 实验一：Role Prompting 的局限性
测试问题：一个班级有36个学生，其中男生比女生多4人。
请问这个班级有多少女生？

学生答案：18人
⚠️ 实际正确答案：16人
💡 陷阱：很多人会直接 36÷2=18，忽略了'男生比女生多4人'这个条件

❌ 方法1：纯 Role Prompting（扮演角色直接判断）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来分析这个问题，判断学生答案是否正确。

---

### **题目：**

一个班级有36个学生，其中男生比女生多4人。  
请问这个班级有多少女生？

---

### **解题过程：**

设女生人数为 $ x $，则男生人数为 $ x + 4 $。

根据题意，总人数为 36，因此：

$$
x + (x + 4) = 36
$$

化简：

$$
2x + 4 = 36
$$

$$
2x = 32
$$

$$
x = 16
$$

---

### **结论：**

女生人数是 **16人**。

---

### **判断学生答案是否正确：**

学生答案是 **18人**，这是 **错误的**。

---

### ✅ **正确答案：16人**  
❌ **学生答案：18人（错误）**

📊 结论: ✓ 正确识别错误


⚠️ 方法2：Role + 简单验证（加入"仔细检查"指令）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来仔细分析这个问题，并判断学生答案是否正确。

---

### **题目：**

一个班级有36个学生，其中男生比女生多4人。  
请问这个班级有多少女生？

---

### **分析过程：**

设女生人数为 $ x $，则男生人数为 $ x + 4 $。

根据题意，总人数为 36，因此可以列出方程：

$$
x + (x + 4) = 36
$$

化简：

$$
2x + 4 = 36
$$

$$
2x = 32
$$

$$
x = 16
$$

---

### **结论：*

---

# 实验一B：质疑推论（测试变体2）

## 📖 任务说明

**任务**：判断一个需要验证推论的逻辑问题

**测试问题**：
> 小明说："我今年12岁，我爸爸今年36岁，所以我爸爸的年龄是我的3倍。
> 再过10年，我22岁，我爸爸46岁，我爸爸的年龄还是我的3倍。"

**学生答案**：小明说的对（这个答案是错的！）

**正确答案**：小明说的不对，再过10年爸爸的年龄不是小明的3倍（46÷22=2.09...）

**陷阱分析**：
- 现在确实是3倍关系（36÷12=3）
- 但年龄差永远是24岁，随着时间推移，倍数关系会改变
- 需要实际验证"再过10年"的推论，不能想当然认为倍数关系不变

In [5]:
# 实验一B：质疑推论

logic_problem_1b = """
小明说："我今年12岁，我爸爸今年36岁，所以我爸爸的年龄是我的3倍。
再过10年，我22岁，我爸爸46岁，我爸爸的年龄还是我的3倍。"
"""

student_answer_1b = "小明说的对"
correct_answer_1b = "小明说的不对（46÷22≈2.09，不是3倍）"

print("="*80)
print("🔬 实验一B：质疑推论")
print("="*80)
print(f"测试问题：{logic_problem_1b.strip()}")
print(f"\n学生答案：{student_answer_1b}")
print(f"⚠️ 实际正确答案：{correct_answer_1b}")
print(f"💡 陷阱：年龄倍数关系会随时间改变，需要实际验证推论")
print("="*80)

# 方法1：❌ 纯 Role Prompting
print("\n❌ 方法1：纯 Role Prompting")
print("─"*80)
role_only_prompt_1b = f"""你是一位逻辑专家。

请判断以下学生答案是否正确：

问题：
{logic_problem_1b}

学生答案：{student_answer_1b}

判断："""

result1_1b = get_completion(role_only_prompt_1b, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result1_1b or '不对' in result1_1b or '2.09' in result1_1b or '2.1' in result1_1b else '✗ 被误导'}")

time.sleep(3)

# 方法2：⚠️ Role + 简单验证
print("\n\n⚠️ 方法2：Role + 简单验证")
print("─"*80)
role_check_prompt_1b = f"""你是一位逻辑专家。

请判断以下学生答案是否正确，并仔细检查你的判断。

问题：
{logic_problem_1b}

学生答案：{student_answer_1b}

请仔细检查后给出判断："""

result2_1b = get_completion(role_check_prompt_1b, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result2_1b or '不对' in result2_1b or '2.09' in result2_1b or '2.1' in result2_1b else '✗ 被误导'}")

time.sleep(3)

# 方法3：✅ System 2 自验证
print("\n\n✅ 方法3：System 2 自验证")
print("─"*80)
system2_prompt_1b = f"""你的任务是判断学生的答案是否正确。

**重要：请严格按照以下步骤操作！**

步骤1：独立推理
- 忽略学生答案
- 验证第一个陈述：现在爸爸年龄是否是小明的3倍？
- 验证第二个陈述：计算10年后的年龄，再计算46÷22是否等于3
- 得出你自己的结论

步骤2：对比验证
- 将你的答案与学生答案对比
- 如果不同，明确指出错误

步骤3：给出判断
- 输出：学生答案是否正确
- 理由：详细说明

---

问题：
{logic_problem_1b}

学生答案：{student_answer_1b}

请开始："""

result3_1b = get_completion(system2_prompt_1b, temperature=0.3)
print(f"\n📊 结论: {'✓ 正确识别错误' if '错' in result3_1b or '不对' in result3_1b or '2.09' in result3_1b or '2.1' in result3_1b else '✗ 被误导'}")

print("\n" + "="*80 + "\n")

🔬 实验一B：质疑推论
测试问题：小明说："我今年12岁，我爸爸今年36岁，所以我爸爸的年龄是我的3倍。
再过10年，我22岁，我爸爸46岁，我爸爸的年龄还是我的3倍。"

学生答案：小明说的对
⚠️ 实际正确答案：小明说的不对（46÷22≈2.09，不是3倍）
💡 陷阱：年龄倍数关系会随时间改变，需要实际验证推论

❌ 方法1：纯 Role Prompting
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来分析小明的说法是否正确。

---

### 一、当前年龄：

- 小明：12岁  
- 爸爸：36岁  
- 36 ÷ 12 = **3** ✅

所以，**当前小明爸爸的年龄是小明的3倍**，这是正确的。

---

### 二、再过10年：

- 小明：12 + 10 = **22岁**  
- 爸爸：36 + 10 = **46岁**  
- 46 ÷ 22 ≈ **2.09** ❌

所以，**再过10年，小明爸爸的年龄不是小明的3倍**，而是约2.09倍。

---

### 三、结论：

小明说：“**再过10年，我爸爸的年龄还是我的3倍**”，这是**错误的**。

---

### ✅ 正确答案：小明说的**不对**。

---

### ✅ 判断：**错误**（学生答案“小明说的对”是错误的）

📊 结论: ✓ 正确识别错误


⚠️ 方法2：Role + 简单验证
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来仔细分析小明的话是否正确。

---

### 一、小明现在的情况：

- 小明今年 **12岁**
- 小明爸爸今年 **36岁**

我们来验证小明说的：“我爸爸的年龄是我的3倍。”

- 小明爸爸的年龄：36岁  
- 小明的年龄：12岁  
- 36 ÷ 12 = **3**

✅ **小明现在说的是对的**。

---

### 二、再过10年的情况：

- 小明的年龄：12 + 10 = **22岁**
- 小明爸爸的年龄：36 + 10 = **46

---

# 实验二：Multi-Agent Verification（多智能体验证）

## 📖 任务说明

**任务**：解决一个容易在多步骤中出错的复杂应用题

**测试问题**：
> 一个水果店进行促销活动。小李购买了苹果和橙子。
> - 苹果原价8元/斤，打7折后每斤便宜了多少钱？
> - 小李买了苹果3斤、橙子5斤
> - 橙子原价6元/斤，买3斤送1斤（实际只需付3斤的钱）
> - 最后使用了满50减10的优惠券
>
> 请问小李一共支付了多少钱？

**陷阱**：
1. **苹果折扣理解错误**：问的是"便宜了多少钱"，但很多人可能算成"现价多少钱"
2. **橙子计算错误**：买3送1，买5斤需要分析清楚（付4斤的钱）
3. **优惠券条件**：满50才能减10，需要先判断是否满足条件
4. **多步骤累积错误**：任何一步出错都会导致最终答案错误

**正确答案**：
- 苹果便宜：8×(1-0.7)=2.4元/斤
- 苹果总价：8×0.7×3=16.8元
- 橙子需付：5斤实际只需付4斤的钱（买3送1，第二个3斤只付2斤）=6×4=24元
- 小计：16.8+24=40.8元
- 不满50元，**不能使用优惠券**
- **最终支付：40.8元**

## 🎯 三种验证策略

我们将实现分层验证的三个Level：

### Level 1：单模型自验证
```
生成答案 → 自己检查一遍 → 输出
```
**问题**：容易对橙子计算出错，或误用优惠券

### Level 2：双模型交叉验证  
```
Model A 生成答案 → Model B 扮演批评者审查 → 指出问题
```
**优势**：批评者可能发现A忽略的细节

### Level 3：Multi-Agent 验证（最强）
```
主 Agent 计算 → 算术验证 Agent 检查计算 → 逻辑验证 Agent 检查条件 → 汇总结论
```
**优势**：
- 算术验证Agent：检查每个乘法、加法是否正确
- 逻辑验证Agent：检查"买3送1"理解是否正确、优惠券条件是否满足

## 💡 预期结果

- Level 1：可能算错橙子（误以为付3斤钱），或错误使用优惠券（忽略满50条件）
- Level 2：能发现一些错误，但可能仍然漏掉优惠券条件
- Level 3：逻辑验证Agent专门检查条件，错误率最低

In [ ]:
# 实验二：Multi-Agent Verification

# 测试问题（多个陷阱）
math_problem = """
一个水果店进行促销活动。小李购买了苹果和橙子。
- 苹果原价8元/斤，打7折后每斤便宜了多少钱？
- 小李买了苹果3斤、橙子5斤
- 橙子原价6元/斤，买3斤送1斤（实际只需付3斤的钱）
- 最后使用了满50减10的优惠券

请问小李一共支付了多少钱？
"""

print("="*80)
print("🔬 实验二：Multi-Agent Verification（多智能体验证）")
print("="*80)
print(f"测试问题：{math_problem.strip()}")
print("\n⚠️ 陷阱提示：")
print("  1. 苹果问题问的是'便宜了多少'，不是'现价多少'")
print("  2. 橙子买3送1，买5斤实际付几斤的钱？")
print("  3. 满50减10，需要先判断是否满足条件")
print("\n✅ 正确答案：40.8元（16.8+24=40.8，不满50不能用券）")
print("="*80)

# Level 1：单模型自验证
print("\n📌 Level 1：单模型自验证")
print("─"*80)
level1_prompt = f"""请解答以下问题，并在给出答案后自己检查一遍计算是否正确。

问题：
{math_problem}

请逐步计算并给出最终答案："""

result_level1 = get_completion(level1_prompt, temperature=0.3)
print(f"\n{'='*80}")

time.sleep(3)

# Level 2：双模型交叉验证
print("\n\n📌 Level 2：双模型交叉验证")
print("─"*80)

# 第一步：生成答案
print("🔹 步骤1：Generator Agent 生成答案")
print("─"*40)
generator_prompt = f"""请解答以下问题，给出详细的计算步骤。

问题：
{math_problem}

请逐步计算："""

answer_from_generator = get_completion(generator_prompt, temperature=0.3, show_output=True)

time.sleep(2)

# 第二步：批评者审查
print("\n\n🔹 步骤2：Critic Agent 审查答案")
print("─"*40)
critic_prompt = f"""你是一位严格的数学审查员。请审查以下答案，指出任何可能的错误或遗漏。

原问题：
{math_problem}

待审查的答案：
{answer_from_generator}

请逐步检查：
1. 苹果的问题理解是否正确？（问的是便宜了多少，不是现价）
2. 橙子的"买3送1"计算是否正确？买5斤实际需要付几斤的钱？
3. 优惠券的使用条件是否满足？（满50才能减10）
4. 最终答案是否正确？

审查结果："""

result_level2 = get_completion(critic_prompt, temperature=0.3)
print(f"\n{'='*80}")

time.sleep(3)

# Level 3：Multi-Agent 验证（最强）
print("\n\n📌 Level 3：Multi-Agent 验证（分工协作）")
print("─"*80)

# Agent 1：主计算 Agent
print("🔹 Agent 1：Main Calculator（主计算器）")
print("─"*40)
main_calculator_prompt = f"""你是主计算 Agent，负责按步骤计算价格。

问题：
{math_problem}

请逐步计算，每一步都明确标注："""

calc_result = get_completion(main_calculator_prompt, temperature=0.3, show_output=True)

time.sleep(2)

# Agent 2：计算验证 Agent
print("\n\n🔹 Agent 2：Arithmetic Verifier（计算验证器）")
print("─"*40)
arithmetic_verifier_prompt = f"""你是计算验证 Agent，只关注数值计算是否正确。

主计算器的结果：
{calc_result}

请验证：
1. 每个乘法/减法的数值是否正确
2. 小数点位置是否正确
3. 重新计算每一步，确认结果

验证结果："""

arith_check = get_completion(arithmetic_verifier_prompt, temperature=0.3, show_output=True)

time.sleep(2)

# Agent 3：逻辑验证 Agent（关键！）
print("\n\n🔹 Agent 3：Logic Verifier（逻辑验证器）- 专门检查条件和理解")
print("─"*40)
logic_verifier_prompt = f"""你是逻辑验证 Agent，只关注问题理解和条件判断是否正确。

问题：
{math_problem}

主计算器的结果：
{calc_result}

请验证以下逻辑：
1. **苹果问题理解**：题目问的是"打7折后每斤便宜了多少钱"，主计算器是否理解正确？
   - 错误理解：算成打折后的现价
   - 正确理解：算便宜的金额（原价-现价）

2. **橙子"买3送1"理解**：小李买了5斤橙子，实际需要付几斤的钱？
   - 买3送1的规则：每买3斤送1斤（即付3斤得4斤）
   - 买5斤的分析：前3斤付钱送1斤（得4斤），剩余1斤需要付钱
   - 正确答案：付4斤的钱

3. **优惠券使用条件**：题目说"满50减10"，主计算器是否检查了满足条件？
   - 需要先计算总价
   - 判断是否≥50元
   - 如果不满50，不能使用优惠券

逻辑验证结果："""

logic_check = get_completion(logic_verifier_prompt, temperature=0.3, show_output=True)

time.sleep(2)

# Agent 4：汇总 Agent
print("\n\n🔹 Agent 4：Aggregator（结果汇总器）")
print("─"*40)
aggregator_prompt = f"""你是汇总 Agent，综合所有验证结果，给出最终结论。

主计算结果：
{calc_result}

计算验证结果：
{arith_check}

逻辑验证结果：
{logic_check}

请汇总：
1. 所有 Agent 的结论是否一致
2. 如果有冲突，哪个是对的
3. 最终答案是多少

最终结论："""

result_level3 = get_completion(aggregator_prompt, temperature=0.3)

print("\n" + "="*80)
print("💡 对比分析：")
print("  Level 1：单模型自查，容易忽略优惠券条件或算错橙子")
print("  Level 2：双模型交叉，Critic 可能发现部分错误")
print("  Level 3：多 Agent 协作，Logic Verifier 专门检查条件，准确率最高")
print("="*80 + "\n")

🔬 实验二：Multi-Agent Verification（多智能体验证）
测试问题：一个水果店进行促销活动。小李购买了苹果和橙子。
- 苹果原价8元/斤，打7折后每斤便宜了多少钱？
- 小李买了苹果3斤、橙子5斤
- 橙子原价6元/斤，买3斤送1斤（实际只需付3斤的钱）
- 最后使用了满50减10的优惠券

请问小李一共支付了多少钱？

⚠️ 陷阱提示：
  1. 苹果问题问的是'便宜了多少'，不是'现价多少'
  2. 橙子买3送1，买5斤实际付几斤的钱？
  3. 满50减10，需要先判断是否满足条件

✅ 正确答案：40.8元（16.8+24=40.8，不满50不能用券）

📌 Level 1：单模型自验证
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 我们来逐步解答这个问题，并在最后检查计算是否正确。

---

### **第一步：苹果的折扣计算**

- 苹果原价：8元/斤  
- 打7折后价格：  
  $$
  8 \times 0.7 = 5.6 \text{元/斤}
  $$
- 每斤便宜了：  
  $$
  8 - 5.6 = 2.4 \text{元/斤}
  $$

✅ **苹果每斤便宜了2.4元。**

---

### **第二步：计算苹果的总价**

- 小李买了3斤苹果  
- 苹果打折后价格是5.6元/斤  
- 苹果总价：  
  $$
  3 \times 5.6 = 16.8 \text{元}
  $$

✅ **苹果总价是16.8元。**

---

### **第三步：橙子的优惠计算**

- 橙子原价：6元/斤  
- 买3斤送1斤（实际只需付3斤的钱）  
- 小李买了5斤橙子  
- 实际只需付3斤的钱  
- 橙子总价：  
  $$
  3 \times 6 = 18 \text{元}
  $$

✅ **橙子总价是18元。**

---

### **第四步：计算总金额（不含优惠券）**

- 苹果总价：16.8元  
- 橙子总价：18元  
- 总金额：  
  $$
  16.8 + 18 = 34.8 \text{元}
  $$

✅ **未使用优惠券前总金额是34.

---

# 🎓 总结与最佳实践

## 核心要点

### 1️⃣ Role Prompting 的本质局限

| 维度 | Role Prompting | System 2 Thinking |
|------|---------------|-------------------|
| **改变内容** | 说话方式（风格） | 思维深度（逻辑） |
| **自我验证** | ❌ 无 | ✅ 有 |
| **错误纠正** | ❌ 难以发现自身错误 | ✅ 多轮验证 |
| **适用场景** | 风格化输出 | 复杂推理任务 |

**结论**：角色扮演不等于深度思考

### 2️⃣ System 2 的两种实现路径

#### 算法层面（模型内置）
- **代表**：OpenAI o1、DeepSeek R1、Qwen3 thinking 模式
- **优势**：内置能力，无需复杂提示
- **实现**：通过 System 2 Distillation 训练

#### 应用层面（提示工程）
- **代表**：Multi-Agent Verification
- **优势**：灵活可控，适用于任何模型
- **实现**：通过精心设计的验证流程

### 3️⃣ Multi-Agent 验证的关键原则

**分离关注点**（Separation of Concerns）：
- 不同 Agent 负责不同维度的验证
- 避免单一 Agent 的盲点
- 通过冲突发现错误

**实践建议**：
```
最小化配置：
  - 简单任务：单模型 + 自验证提示词
  - 中等任务：双模型交叉验证
  - 复杂任务：多 Agent 分工验证

成本考虑：
  - Level 3 Multi-Agent 会增加 3-5 倍 API 调用
  - 仅在高价值任务上使用
  - 可以先用 Level 1/2，失败时升级到 Level 3
```

## 🚀 延伸学习

### 相关技术
- **Self-Consistency**：多次采样取最一致的答案
- **Debate**：多个 Agent 辩论达成共识
- **Constitutional AI**：通过规则自我修正
- **Reflexion**：从错误中学习并改进

### 推理模型对比
| 模型 | 内置思考 | 开源 | 成本 |
|------|---------|------|------|
| OpenAI o1 | ✅ | ❌ | 高 |
| DeepSeek R1 | ✅ | ✅ | 低 |
| Qwen3 (thinking) | ✅ | ✅ | 低 |
| Claude Opus | ❌ (Extended Thinking) | ❌ | 高 |

## 💡 实践指南

### 何时使用 System 2 Thinking？

**✅ 适用场景**：
- 复杂逻辑推理
- 多步骤数学计算
- 需要验证的任务（如审查代码、检查答案）
- 容易出错的任务

**❌ 不适用场景**：
- 简单事实性问答
- 创意写作（风格优先）
- 实时对话（速度优先）
- 信息检索

### 如何选择验证策略？

```python
if 任务简单且容错率高:
    直接生成答案  # 无需验证
elif 任务中等复杂:
    使用 Level 1 自验证  # 成本最低
elif 任务复杂但不关键:
    使用 Level 2 双模型  # 平衡效果和成本
elif 任务关键且复杂:
    使用 Level 3 Multi-Agent  # 最高准确率
```

---

**🎉 恭喜完成 System 2 Thinking 实验！**

你已经掌握了从 Role Prompting 到 Multi-Agent Verification 的完整技术栈！🚀